In [ ]:
!pip install tensorflow tensorflow_addons --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
!pip install transformers --upgrade --quiet
!pip install git+https://github.com/wandb/wandb.git
!yes | apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2


import os



os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['WANDB_API_KEY'] = 'd01b92fa0118a7bba3d1f90cfda3e30f3addf78f'


import random
import numpy as np
from glob import glob
from functools import partial
from PIL import Image, ImageOps

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

from transformers.tf_utils import shape_list

import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint


AUTOTUNE = tf.data.AUTOTUNE

  Cloning https://github.com/wandb/wandb.git to /tmp/pip-req-build-0tr5xubf
  Running command git clone --filter=blob:none --quiet https://github.com/wandb/wandb.git /tmp/pip-req-build-0tr5xubf
  Resolved https://github.com/wandb/wandb.git to commit 6b8a4dbde1e120784527ab60b8985be73d8383da
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00
  Created wheel for wandb: filename=wandb-0.16.5.dev1-py3-none-any.whl size=2202783 sha256=57b35ff42df787fee82c123daa5133105b2fc0c592e6210f8c1903b63f5d8777
  Stored in directory: /tmp/pip-ephem-wheel-cache-i2co2iq9/wheels/d4/e8/45/c3dcab816be8169b14b0f8f9da0813be53e067c27980153a1f
Succ

In [ ]:
wandb.init(project="mirnet-v2",entity="nemesis_xvi", job_type="colab-train")

strategy = tf.distribute.MirroredStrategy()

config = wandb.config

config.seed = 42
random.seed(config.seed)
tf.random.set_seed(config.seed)

config.num_gpus = len(tf.config.list_physical_devices('GPU'))
config.dataset_artifact_address = 'nemesis_xvi/mirnet-v2/lol-dataset:v0'
config.image_size = 128
config.max_train_images = 400
config.batch_size_per_replica = 4
config.batch_size = config.batch_size_per_replica * strategy.num_replicas_in_sync

config.channels = 80
config.channel_factor = 1.5
config.num_mrb_blocks = 2
config.add_residual_connection = True

config.initial_learning_rate = 2e-4
config.minimum_learning_rate = 1e-6
config.epochs = 100

wandb: Currently logged in as: chesslover-201. Use `wandb login --relogin` to force relogin


Problem at: <ipython-input-4-99e6113214e5> 1 <cell line: 1>


CommError: It appears that you do not have permission to access the requested resource. Please reach out to the project owner to grant you access. If you have the correct permissions, verify that there are no issues with your networking setup.(Error 403: Forbidden)

In [ ]:
!mkdir LOLdataset

In [ ]:
!unzip '/content/drive/MyDrive/LOLdataset.zip' -d '/content/LOLdataset'

In [ ]:
dataset_dir = "/content/LOLdataset"
TRAIN_LOW_LIGHT_IMAGES = sorted(
    glob(os.path.join(dataset_dir, "our485/low/*"))
)[:config.max_train_images]
TRAIN_GROUND_TRUTH_IMAGES = sorted(
    glob(os.path.join(dataset_dir, "our485/high/*"))
)[:config.max_train_images]

VAL_LOW_LIGHT_IMAGES = sorted(
    glob(os.path.join(dataset_dir, "our485/low/*"))
)[config.max_train_images:]
VAL_GROUND_TRUTH_IMAGES = sorted(
    glob(os.path.join(dataset_dir, "our485/high/*"))
)[config.max_train_images:]

TEST_LOW_LIGHT_IMAGES = sorted(
    glob(os.path.join(dataset_dir, "eval15/low/*"))
)
TEST_GROUND_TRUTH_IMAGES = sorted(
    glob(os.path.join(dataset_dir, "eval15/high/*"))
)

print(f"Train Dataset: ({len(TRAIN_LOW_LIGHT_IMAGES)}, {len(TRAIN_GROUND_TRUTH_IMAGES)})")
print(f"Validation Dataset: ({len(VAL_LOW_LIGHT_IMAGES)}, {len(VAL_GROUND_TRUTH_IMAGES)})")
print(f"Test Dataset: ({len(TEST_LOW_LIGHT_IMAGES)}, {len(TEST_GROUND_TRUTH_IMAGES)})")

In [ ]:
def read_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.cast(image, dtype=tf.float32) / 255.0
    return image


def random_crop(low_image, gt_image):
    low_image_shape = tf.shape(low_image)[:2]
    crop_width = tf.random.uniform(
        shape=(), maxval=low_image_shape[1] - config.image_size + 1, dtype=tf.int32
    )
    crop_height = tf.random.uniform(
        shape=(), maxval=low_image_shape[0] - config.image_size + 1, dtype=tf.int32
    )
    low_image_cropped = low_image[
        crop_height : crop_height + config.image_size,
        crop_width : crop_width + config.image_size
    ]
    gt_image_cropped = gt_image[
        crop_height : crop_height + config.image_size,
        crop_width : crop_width + config.image_size
    ]
    low_image_cropped.set_shape([config.image_size, config.image_size, 3])
    gt_image_cropped.set_shape([config.image_size, config.image_size, 3])
    return low_image_cropped, gt_image_cropped


def resize_images(low_image, gt_image):
    low_image = tf.image.resize(low_image, size=[config.image_size, config.image_size])
    gt_image = tf.image.resize(gt_image, size=[config.image_size, config.image_size])
    low_image.set_shape([config.image_size, config.image_size, 3])
    gt_image.set_shape([config.image_size, config.image_size, 3])
    return low_image, gt_image


def load_data(low_light_image_path, enhanced_image_path, apply_resize):
    low_light_image = read_image(low_light_image_path)
    enhanced_image = read_image(enhanced_image_path)
    low_light_image, enhanced_image = (
        resize_images(low_light_image, enhanced_image) if apply_resize
        else random_crop(low_light_image, enhanced_image)
    )
    return low_light_image, enhanced_image


def get_dataset(low_light_images, enhanced_images, apply_resize):
    dataset = tf.data.Dataset.from_tensor_slices((low_light_images, enhanced_images))
    dataset = dataset.map(
        partial(load_data, apply_resize=apply_resize),
        num_parallel_calls=AUTOTUNE
    )
    dataset = dataset.batch(config.batch_size, drop_remainder=True)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
train_dataset = get_dataset(TRAIN_LOW_LIGHT_IMAGES, TRAIN_GROUND_TRUTH_IMAGES, apply_resize=False)
val_dataset = get_dataset(VAL_LOW_LIGHT_IMAGES, VAL_GROUND_TRUTH_IMAGES, apply_resize=True)

print("Train Dataset:", train_dataset.element_spec)
print("Validation Dataset:", val_dataset.element_spec)

In [ ]:
class SelectiveKernelFeatureFusion(tf.keras.layers.Layer):
    def __init__(self, channels: int, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.hidden_channels = max(int(channels / 8), 4)

        self.average_pooling = tfa.layers.AdaptiveAveragePooling2D(output_size=1)

        self.conv_channel_downscale = tf.keras.layers.Conv2D(
            self.hidden_channels, kernel_size=1, padding="same"
        )
        self.conv_attention_1 = tf.keras.layers.Conv2D(
            channels, kernel_size=1, strides=1, padding="same"
        )
        self.conv_attention_2 = tf.keras.layers.Conv2D(
            channels, kernel_size=1, strides=1, padding="same"
        )

        self.leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)
        self.softmax = tf.keras.layers.Softmax(axis=-1)

    def call(self, inputs, *args, **kwargs):
        combined_input_features = inputs[0] + inputs[1]
        channel_wise_statistics = self.average_pooling(combined_input_features)
        downscaled_channel_wise_statistics = self.conv_channel_downscale(
            channel_wise_statistics
        )
        attention_vector_1 = self.softmax(
            self.conv_attention_1(downscaled_channel_wise_statistics)
        )
        attention_vector_2 = self.softmax(
            self.conv_attention_2(downscaled_channel_wise_statistics)
        )
        selected_features = (
            inputs[0] * attention_vector_1 + inputs[1] * attention_vector_2
        )
        return selected_features

In [ ]:
class ContextBlock(tf.keras.layers.Layer):
    def __init__(self, channels: int, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.mask_conv = tf.keras.layers.Conv2D(
            1, kernel_size=1, padding="same"
        )

        self.channel_add_conv_1 = tf.keras.layers.Conv2D(
            channels, kernel_size=1, padding="same"
        )
        self.channel_add_conv_2 = tf.keras.layers.Conv2D(
            channels, kernel_size=1, padding="same"
        )

        self.softmax = tf.keras.layers.Softmax(axis=1)
        self.leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)

    def modeling(self, inputs):
        _, height, width, channels = shape_list(inputs)
        reshaped_inputs = tf.expand_dims(
            tf.reshape(inputs, (-1, channels, height * width)), axis=1
        )

        context_mask = self.mask_conv(inputs)
        context_mask = tf.reshape(context_mask, (-1, height * width, 1))
        context_mask = self.softmax(context_mask)
        context_mask = tf.expand_dims(context_mask, axis=1)

        context = tf.reshape(
            tf.matmul(reshaped_inputs, context_mask), (-1, 1, 1, channels)
        )
        return context

    def call(self, inputs, *args, **kwargs):
        context = self.modeling(inputs)
        channel_add_term = self.channel_add_conv_1(context)
        channel_add_term = self.leaky_relu(channel_add_term)
        channel_add_term = self.channel_add_conv_2(channel_add_term)
        return inputs + channel_add_term


class ResidualContextBlock(tf.keras.layers.Layer):
    def __init__(self, channels: int, groups: int, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.conv_1 = tf.keras.layers.Conv2D(
            channels, kernel_size=3, padding="same", groups=groups
        )
        self.conv_2 = tf.keras.layers.Conv2D(
            channels, kernel_size=3, padding="same", groups=groups
        )
        self.leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)

        self.context_block = ContextBlock(channels=channels)

    def call(self, inputs):
        x = self.conv_1(inputs)
        x = self.leaky_relu(x)
        x = self.conv_2(x)
        x = self.context_block(x)
        x = self.leaky_relu(x)
        x = x + inputs
        return x

In [ ]:
class DownBlock(tf.keras.layers.Layer):
    def __init__(
        self, channels: int, channel_factor: float, *args, **kwargs
    ):
        super(DownBlock, self).__init__(*args, **kwargs)
        self.average_pool = tf.keras.layers.AveragePooling2D(
            pool_size=2, strides=2
        )
        self.conv = tf.keras.layers.Conv2D(
            int(channels * channel_factor),
            kernel_size=1,
            strides=1,
            padding="same"
        )

    def call(self, inputs, *args, **kwargs):
        return self.conv(self.average_pool(inputs))


class DownSampleBlock(tf.keras.layers.Layer):
    def __init__(
        self,
        channels: int,
        scale_factor: int,
        channel_factor: float,
        *args,
        **kwargs
    ):
        super(DownSampleBlock, self).__init__(*args, **kwargs)
        self.layers = []
        for _ in range(int(np.log2(scale_factor))):
            self.layers.append(DownBlock(channels, channel_factor))
            channels = int(channels * channel_factor)

    def call(self, x, *args, **kwargs):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
class UpBlock(tf.keras.layers.Layer):
    def __init__(self, channels: int, channel_factor: float, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.conv = tf.keras.layers.Conv2D(
            int(channels // channel_factor), kernel_size=1, strides=1, padding="same"
        )
        self.upsample = tf.keras.layers.UpSampling2D(size=2, interpolation="bilinear")

    def call(self, inputs, *args, **kwargs):
        return self.upsample(self.conv(inputs))


class UpSampleBlock(tf.keras.layers.Layer):
    def __init__(
        self, channels: int, scale_factor: int, channel_factor: float, *args, **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.layers = []
        for _ in range(int(np.log2(scale_factor))):
            self.layers.append(UpBlock(channels, channel_factor))
            channels = int(channels // channel_factor)

    def call(self, x, *args, **kwargs):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
class MultiScaleResidualBlock(tf.keras.layers.Layer):
    def __init__(
        self,
        channels: int,
        channel_factor: float,
        groups: int,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)

        # Residual Context Blocks
        self.rcb_top = ResidualContextBlock(
            int(channels * channel_factor**0), groups=groups
        )
        self.rcb_middle = ResidualContextBlock(
            int(channels * channel_factor**1), groups=groups
        )
        self.rcb_bottom = ResidualContextBlock(
            int(channels * channel_factor**2), groups=groups
        )

        # Downsample Blocks
        self.down_2 = DownSampleBlock(
            channels=int((channel_factor**0) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )
        self.down_4_1 = DownSampleBlock(
            channels=int((channel_factor ** 0) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )
        self.down_4_2 = DownSampleBlock(
            channels=int((channel_factor ** 1) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )

        # UpSample Blocks
        self.up21_1 = UpSampleBlock(
            channels=int((channel_factor ** 1) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )
        self.up21_2 = UpSampleBlock(
            channels=int((channel_factor ** 1) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )
        self.up32_1 = UpSampleBlock(
            channels=int((channel_factor ** 2) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )
        self.up32_2 = UpSampleBlock(
            channels=int((channel_factor ** 2) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )

        # SKFF Blocks
        self.skff_top = SelectiveKernelFeatureFusion(
            channels=int(channels * channel_factor**0)
        )
        self.skff_middle = SelectiveKernelFeatureFusion(
            channels=int(channels * channel_factor**1)
        )

        # Convolution
        self.conv_out = tf.keras.layers.Conv2D(
            channels, kernel_size=1, padding="same"
        )

    def call(self, inputs, *args, **kwargs):
        x_top = inputs
        x_middle = self.down_2(x_top)
        x_bottom = self.down_4_2(self.down_4_1(x_top))

        x_top = self.rcb_top(x_top)
        x_middle = self.rcb_middle(x_middle)
        x_bottom = self.rcb_bottom(x_bottom)

        x_middle = self.skff_middle([x_middle, self.up32_1(x_bottom)])
        x_top = self.skff_top([x_top, self.up21_1(x_middle)])

        x_top = self.rcb_top(x_top)
        x_middle = self.rcb_middle(x_middle)
        x_bottom = self.rcb_bottom(x_bottom)

        x_middle = self.skff_middle([x_middle, self.up32_2(x_bottom)])
        x_top = self.skff_top([x_top, self.up21_2(x_middle)])

        output = self.conv_out(x_top)
        output = output + inputs

        return output

In [ ]:
class RecursiveResidualGroup(tf.keras.layers.Layer):
    def __init__(
        self,
        channels: int,
        num_mrb_blocks: int,
        channel_factor: float,
        groups: int,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.layers = [
            MultiScaleResidualBlock(channels, channel_factor, groups)
            for _ in range(num_mrb_blocks)
        ]
        self.layers.append(
            tf.keras.layers.Conv2D(
                channels, kernel_size=3, strides=1, padding="same"
            )
        )

    def call(self, inputs, *args, **kwargs):
        residual = inputs
        for layer in self.layers:
            residual = layer(residual)
        residual = residual + inputs
        return residual


class MirNetv2(tf.keras.Model):
    def __init__(
        self,
        channels: int,
        channel_factor: float,
        num_mrb_blocks: int,
        add_residual_connection: bool,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)

        self.add_residual_connection = add_residual_connection

        self.conv_in = tf.keras.layers.Conv2D(
            channels, kernel_size=3, padding="same"
        )

        self.rrg_block_1 = RecursiveResidualGroup(
            channels, num_mrb_blocks, channel_factor, groups=1
        )
        self.rrg_block_2 = RecursiveResidualGroup(
            channels, num_mrb_blocks, channel_factor, groups=2
        )
        self.rrg_block_3 = RecursiveResidualGroup(
            channels, num_mrb_blocks, channel_factor, groups=4
        )
        self.rrg_block_4 = RecursiveResidualGroup(
            channels, num_mrb_blocks, channel_factor, groups=4
        )

        self.conv_out = tf.keras.layers.Conv2D(
            3, kernel_size=3, padding="same"
        )

    def call(self, inputs, training=None, mask=None):
        shallow_features = self.conv_in(inputs)
        deep_features = self.rrg_block_1(shallow_features)
        deep_features = self.rrg_block_2(deep_features)
        deep_features = self.rrg_block_3(deep_features)
        deep_features = self.rrg_block_4(deep_features)
        output = self.conv_out(deep_features)
        output = output + inputs if self.add_residual_connection else output
        return output

In [ ]:
class CharbonnierLoss(tf.keras.losses.Loss):
    def __init__(self, epsilon: float, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.epsilon = tf.convert_to_tensor(epsilon)

    def call(self, y_true, y_pred):
        squared_difference = tf.square(y_true - y_pred)
        return tf.reduce_mean(
            tf.sqrt(squared_difference + tf.square(self.epsilon))
        )


class PSNRMetric(tf.keras.metrics.Metric):
    def __init__(self, max_val: float, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.max_val = max_val
        self.psnr = tf.keras.metrics.Mean(name="psnr")

    def update_state(self, y_true, y_pred, *args, **kwargs):
        psnr = tf.image.psnr(y_true, y_pred, max_val=self.max_val)
        self.psnr.update_state(psnr, *args, **kwargs)

    def result(self):
        return self.psnr.result()

    def reset_state(self):
        self.psnr.reset_state()


class SSIMMetric(tf.keras.metrics.Metric):
    def __init__(self, max_val: float, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.max_val = max_val
        self.ssim = tf.keras.metrics.Mean(name="ssim")

    def update_state(self, y_true, y_pred, *args, **kwargs):
        ssim = tf.image.ssim(y_true, y_pred, max_val=self.max_val)
        self.ssim.update_state(ssim, *args, **kwargs)

    def result(self):
        return self.ssim.result()

    def reset_state(self):
        self.ssim.reset_state()

In [ ]:
with strategy.scope():
    model = MirNetv2(
        channels=config.channels,
        channel_factor=config.channel_factor,
        num_mrb_blocks=config.num_mrb_blocks,
        add_residual_connection=config.add_residual_connection
    )

    dummy_inputs = tf.ones((1, config.image_size, config.image_size, 3))
    dummy_outputs = model(dummy_inputs)
    model.summary(expand_nested=True)
    print("\nInput Shape:", dummy_inputs.shape)
    print("Output Shape:", dummy_outputs.shape)

    loss = CharbonnierLoss(epsilon=1e-3)

    psnr_metric = PSNRMetric(max_val=1.0)
    ssim_metric = SSIMMetric(max_val=1.0)

    decay_steps = (config.max_train_images // config.batch_size) * config.epochs
    lr_schedule_fn = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=config.initial_learning_rate,
        decay_steps=decay_steps,
        alpha=config.minimum_learning_rate,
    )
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=lr_schedule_fn, beta_1=0.9, beta_2=0.999,
    )

    model.compile(
        optimizer=optimizer, loss=loss, metrics=[psnr_metric, ssim_metric]
    )

In [ ]:
%%wandb

callbacks = [
    WandbMetricsLogger(),
    WandbModelCheckpoint(filepath="model", save_best_only=False)
]

model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=config.epochs,
    callbacks=callbacks
)